In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import math
import os.path
from os.path import join
import numpy as np
import imodelsx
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import scipy.stats
import imodelsx.viz
import json
import seaborn as sns
import data
import joblib
from matplotlib.gridspec import GridSpec
from scipy.stats import spearmanr
LEVELS = ['Very Negative', 'Negative', 'Neutral',
          'No response', 'Positive', 'Very Positive']
files_dict = data.load_files_dict_single_site()

### Filter data for single-site analysis

In [ ]:
# checkpoint = 'meta-llama/Llama-2-7b-hf'
# checkpoint = 'meta-llama/Llama-2-70b-hf'
checkpoint = 'gpt-4'  # gpt-35-turbo
# checkpoint = 'gpt-35-turbo'
# checkpoint = 'mistralai/Mistral-7B-v0.1'


# site = 'Atlanta'
# site = 'Columbus'
# site = 'WashingtonDC'
site = 'Charlotte'  # Houston, Portland
df = files_dict[site]
qs, responses_df, themes_df = data.split_single_site_df(df)

### Run sentiment
Note: this uses a lot of API calls (num questions * num responses), maybe around 400

In [ ]:
def run_sentiment(site, checkpoint, qs, responses_df, themes_df, llm=None):
    sentiment_prompt = '''### You are given a question and a response. Rate the sentiment/supportiveness of the response on a scale of 1 to 5, where 1 is very negative and 5 is very positive. ###

Question: {question}

Response: {response}

Rating (1-5):'''
    if llm is None:
        llm = imodelsx.llm.get_llm(checkpoint, repeat_delay=None)

    num_questions = len(qs)
    sentiments = defaultdict(list)
    for question_num in tqdm(range(num_questions), position=0):
        question, responses, theme_dict = data.get_data_for_question_single_site(
            question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)

        for response_num in tqdm(range(len(responses)), position=1):
            response = responses.values[response_num]

            if pd.isna(response):
                sentiments[question_num].append(np.nan)
            else:
                prompt = sentiment_prompt.format(
                    question=question, response=response)
                ans = llm(prompt)
                sentiments[question_num].append(ans)

    # save
    pd.DataFrame(sentiments).T.to_pickle(
        join(data.PROCESSED_DIR, f'sentiments_df_{site}_{checkpoint.split("/")[-1]}.pkl'))
    return sentiments


def compute_and_save_sent_df(sentiments, site, checkpoint):

    sent_df = pd.DataFrame([(key, var) for (key, L) in sentiments.items() for var in L],
                           columns=['Question', 'Value'])

    # round  values
    sent_df['Value'] = sent_df['Value'].astype(float).round()
    value_maps = {
        1: 'Very Negative',
        2: 'Negative',
        3: 'Neutral',
        4: 'Positive',
        5: 'Very Positive',
    }
    sent_df['Value'] = sent_df['Value'].map(value_maps.get)
    sent_df['Value'] = sent_df['Value'].fillna('No response')

    sent_df = sent_df.groupby(
        ['Question', 'Value']).size().unstack(fill_value=0)
    sent_df = sent_df.reindex(LEVELS, axis=1)
    if checkpoint == 'gpt-4':
        joblib.dump(sent_df, join(data.PROCESSED_DIR,
                    f'sent_df_{site}_gpt-4.pkl'))
    return sent_df

# run single
# sentiments = run_sentiment(site, checkpoint, qs, responses_df, themes_df)


# run all sites
llm = imodelsx.llm.get_llm(checkpoint, repeat_delay=20)
# ['Atlanta', 'Columbus', 'WashingtonDC']:
# for site in ['Charlotte', 'Houston', 'Portland']:
for site in ['Dallas', 'Seattle', 'Tucson']:
    df = files_dict[site]
    qs, responses_df, themes_df = data.split_single_site_df(df)
    sentiments = run_sentiment(
        site, checkpoint, qs, responses_df, themes_df, llm=llm)
    sent_df = compute_and_save_sent_df(sentiments, site, checkpoint)

### Plot sentiment

In [ ]:
# make plot
sent_df = compute_and_save_sent_df(sentiments, site, checkpoint)
sent_df = sent_df.sort_values(by=LEVELS, ascending=False)
colors = sns.diverging_palette(20, 220, n=6).as_hex()
colors = colors[:2] + ['#ddd', '#eee'] + colors[-2:]
sent_df.plot(kind='barh', stacked=True, figsize=(5, 10), color=colors)

plt.yticks(range(46), labels=df['Domain'].values[sent_df.index.values])
plt.legend(bbox_to_anchor=(0.5, 1.1), loc='center', ncol=3, title='Sentiment')
plt.xlabel('Answer count')
plt.ylabel('Question number and domain')
plt.title(site)
plt.savefig(f'../figs/eda/sentiment_example_{site}.pdf', bbox_inches='tight')
plt.show()